In [1]:
import os
import time
import random
import requests
import import_ipynb
import Cicero as Cic
import pandas as pnd
import Glossa as glos
import BggHelper as BH
import BggDbGetter as BDG
import BggDbScraper as BDS
from langdetect import detect
from tqdm.notebook import tqdm
from boardgamegeek import BoardGameGeek

importing Jupyter notebook from Cicero.ipynb
importing Jupyter notebook from BggHelper.ipynb
importing Jupyter notebook from BggDbGetter.ipynb
importing Jupyter notebook from BggDbScraper.ipynb
importing Jupyter notebook from Glossa.ipynb


# Step 1

Access BBG API's to download game and user preferences data.
Define criterion used for setting the parameters that drive the download in termis of the variables defining the dataset, such collecting games only for a specified period or a specified set of categories. The wider the game collection, the better.

In [2]:
helper = BH.BggHelper()
new_df = helper.bgg_start(get=False, scrape=False)

#### Dataframe reading

In [3]:
df = pnd.read_csv("BGG_sampled.csv")
df.shape
dfh = df

In [4]:
df.columns

Index(['Unnamed: 0', 'boardgame_rank', 'id', 'families', 'categories',
       'mechanics', 'designers', 'artists', 'publishers', 'description',
       'yearpublished', 'minplayers', 'maxplayers', 'playingtime', 'minage',
       'name', 'usersrated', 'average', 'bayesaverage', 'stddev', 'owned',
       'numweights', 'averageweight', 'ranks'],
      dtype='object')

In [5]:
gls = glos.Glossa(dfh, "en")
cicero = Cic.Cicero(gls)

In [6]:
#gls.get_interesting_fields(["name", "usersrated", "description"])

In [7]:
#stub = cicero.clean_lemming("description")
cicero = cicero.clean_lemming("description")

Glossa.ipynb:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "import pandas as pnd\n",


Glossa.ipynb:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "from nltk.stem import LancasterStemmer, WordNetLemmatizer\n",
Glossa.ipynb:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "from nltk.corpus import stopwords\n",


In [8]:
#stub = cicero.eng_lemming("name")
cicero = cicero.eng_lemming("name")

In [9]:
#stub = cicero.clean_lemming("families")
cicero = cicero.clean_lemming("families")

In [ ]:
#stub["name"]

In [ ]:
#stub["description"].value_counts()
#glos.Glossa(stub, "en").get_interesting_fields(["name", "families", "description"])

In [10]:
model = cicero.word2vec_train(["name", "families", "description"])

In [11]:
model["gloomhaven"]

/home/tore/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.21467204,  0.10683513, -0.09037238, -0.06441708,  0.11019051,
       -0.32280302, -0.19992922, -0.10190924,  0.17337987,  0.18681447,
        0.22181189, -0.16061054, -0.10725777,  0.02592668, -0.07624428,
        0.1575759 , -0.09080415,  0.06267717, -0.03167302, -0.13378909,
       -0.26961964,  0.2568054 ,  0.06759249, -0.1320302 ,  0.12707536,
        0.18337923,  0.06327916, -0.26420793, -0.14840323, -0.07681322,
        0.00981143,  0.09535832,  0.28861603,  0.36572874, -0.0715105 ,
        0.01140393,  0.18209966,  0.03602567, -0.03950054,  0.17510588,
        0.00504446, -0.02765262,  0.01316215, -0.07342527,  0.14969975,
       -0.03208447,  0.13279854,  0.24165411, -0.14197123, -0.00855118,
       -0.12246997, -0.04831386,  0.20756908,  0.07154438, -0.0320094 ,
        0.12471969, -0.02655767, -0.01359473, -0.03675269, -0.1221869 ,
        0.11106967,  0.0351949 ,  0.08740759,  0.01544129, -0.19531967,
       -0.17482688, -0.06493781, -0.00740329,  0.11161916,  0.08

In [ ]:
#x

In [ ]:
"z" < "ð"

bag = dict()
for row in stub["description"]:
    b = set(row)
    for item in b:
        if item.isalpha():
            bag[item] = row.count(item)
sorted(bag.items(), key=lambda x: x[0], reverse=True)

import string
string.punctuation


# Step 2
Pre-process game text (title and description) in order to define the terms to include in the variables to be observed, eventually using a word embedding model to normalize and/or expand the terminology extracted from the game description.

## Follow this url:
### https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92

There is the question of HOW to train the word embedding model and how to calculate the similarity between most and least successful games. <br>
Maybe adding a dataset column is not such a bad idea, keeps it clean and of easy computation. <br>
Glossa and Cicero work relatively well, now a new library for this analysis is needed. Might call it Pitagora. Don't know

#### Need to create a column in the dataframe containing all the important words in the row. <br> This takes into account Name, description, mechanics and publishers

In [ ]:
df.columns

# TODO: Get most frequent words from description, discard others

# TODO: Make class to clean dataframe and keep only selected language (English) descriptions : DONE

### Word normalization time
For each entry of the dataset the following must be done:
<ol>
    <li>Clean dataframe from shit entries [Done]</li>
    <li>Take description field</li>
    <li>Remove stopwords</li>
    <li>Stem words</li>
    <li>Create normalized description with word embedding model</li>
    <li>Override description field (or create new Dataset)</li>
</ol>

langs = list()
for row in tqdm(list(df.iterrows())):
    try:
        langs.append(detect(row[1]["description"]))
    except:
        pass

set_langs = set(langs)
for i in set_langs:
    print("{0}: {1}".format(i, langs.count(i)))

df.head()
#df[df.name == "Gloomhaven"]
#df[df.name != "Gloomhaven"]

## Step 3
Perform a correlation analysis of the variables. On top of this, try to design and implement a model for predicting the "subjective" variable values starting from the "objective" variables. Optionally, define a recommendation system for games.

## Step 4
Include in the paper a report describing the output of Step 3